# Projeto 16: Autoencoders e classificação

In [1]:
!pip install torch==1.4.0 torchvision==0.5.0

     |████████████████████████████████| 753.4MB 21kB/s 
     |████████████████████████████████| 4.0MB 49.1MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


## Etapa 1: Importação das bibliotecas

In [2]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score

In [3]:
torch.manual_seed(123)

## Etapa 2: Base de dados

In [4]:
data_train = datasets.MNIST('MNIST-data', train = True, download=True,
                               transform = transforms.ToTensor())
loader_train = torch.utils.data.DataLoader(data_train, batch_size=256,
                                           shuffle = True)

Extracting MNIST-data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST-data/MNIST/raw


Extracting MNIST-data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST-data/MNIST/raw


Extracting MNIST-data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST-data/MNIST/raw


Extracting MNIST-data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST-data/MNIST/raw
Processing...
Done!


In [5]:
data_test = datasets.MNIST('MNIST-data', train = False, download=True,
                           transform = transforms.ToTensor())
loader_test = torch.utils.data.DataLoader(data_test, batch_size=256,
                                          shuffle=True)

## Etapa 3: Construção do autoencoder

In [6]:
class autoencoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.dense0 = nn.Linear(in_features=784, out_features=32)
    self.activation0 = nn.ReLU()
    self.dense1 = nn.Linear(32,784)
    self.activation1 = nn.Sigmoid()

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    return X

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [8]:
model = autoencoder()
model.to(device)

autoencoder(
  (dense0): Linear(in_features=784, out_features=32, bias=True)
  (activation0): ReLU()
  (dense1): Linear(in_features=32, out_features=784, bias=True)
  (activation1): Sigmoid()
)

In [11]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

## Etapa 4: Treinamento do modelo

In [12]:
for epoch in range(20):
  running_loss_train = 0.
  for data in loader_train:
    model.train()
    inputs, _ = data
    inputs = inputs.to(device)
    inputs = inputs.view(-1, 28*28)

    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, inputs.view(*outputs.shape))
    loss.backward()

    optimizer.step()

    running_loss_train += loss.item()

  running_loss_val = 0.
  for data in loader_test:
    model.eval()
    inputs, _ = data
    inputs = inputs.to(device)
    inputs = inputs.view(-1,28*28)
    
    outputs = model(inputs)
    loss = criterion(outputs, inputs.view(*outputs.shape))
    running_loss_val += loss.item()

  print('ÉPOCA {:3d}: PERDA TREINO {:.5f} PERDA VALIDAÇÃO {:.5f}' \
        .format(epoch+1, running_loss_train/len(loader_train), running_loss_val/len(loader_test)))

ÉPOCA   1: PERDA TREINO 0.29703 PERDA VALIDAÇÃO 0.20237
ÉPOCA   2: PERDA TREINO 0.18216 PERDA VALIDAÇÃO 0.16382
ÉPOCA   3: PERDA TREINO 0.15509 PERDA VALIDAÇÃO 0.14599
ÉPOCA   4: PERDA TREINO 0.14117 PERDA VALIDAÇÃO 0.13517
ÉPOCA   5: PERDA TREINO 0.13216 PERDA VALIDAÇÃO 0.12711
ÉPOCA   6: PERDA TREINO 0.12553 PERDA VALIDAÇÃO 0.12166
ÉPOCA   7: PERDA TREINO 0.12053 PERDA VALIDAÇÃO 0.11690
ÉPOCA   8: PERDA TREINO 0.11689 PERDA VALIDAÇÃO 0.11407
ÉPOCA   9: PERDA TREINO 0.11434 PERDA VALIDAÇÃO 0.11195
ÉPOCA  10: PERDA TREINO 0.11261 PERDA VALIDAÇÃO 0.11056
ÉPOCA  11: PERDA TREINO 0.11141 PERDA VALIDAÇÃO 0.10932
ÉPOCA  12: PERDA TREINO 0.11064 PERDA VALIDAÇÃO 0.10881
ÉPOCA  13: PERDA TREINO 0.11008 PERDA VALIDAÇÃO 0.10857
ÉPOCA  14: PERDA TREINO 0.10975 PERDA VALIDAÇÃO 0.10802
ÉPOCA  15: PERDA TREINO 0.10948 PERDA VALIDAÇÃO 0.10766
ÉPOCA  16: PERDA TREINO 0.10927 PERDA VALIDAÇÃO 0.10746
ÉPOCA  17: PERDA TREINO 0.10904 PERDA VALIDAÇÃO 0.10659
ÉPOCA  18: PERDA TREINO 0.10740 PERDA VALIDAÇÃO 

## Etapa 5: Geração do encoder

In [13]:
list(model.children())

[Linear(in_features=784, out_features=32, bias=True),
 ReLU(),
 Linear(in_features=32, out_features=784, bias=True),
 Sigmoid()]

In [14]:
class encoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.dense0 = list(model.children())[0]
    self.activation0 = list(model.children())[1]

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    return X

In [15]:
model_enc = encoder()
model_enc.to(device)

encoder(
  (dense0): Linear(in_features=784, out_features=32, bias=True)
  (activation0): ReLU()
)

## Etapa 6: Codificação dos previsores

In [16]:
X_train = data_train.data.float() / 255
X_train = X_train.view(-1,784)
X_train = X_train.to(device)

In [17]:
X_train.shape

torch.Size([60000, 784])

In [18]:
with torch.no_grad():
  X_train_cod = model_enc(X_train)
cls_train = data_train.targets

In [19]:
X_train_cod.shape

torch.Size([60000, 32])

In [20]:
cls_train

tensor([5, 0, 4,  ..., 5, 6, 8])

In [21]:
data_train_cod = torch.utils.data.TensorDataset(X_train_cod, cls_train)
loader_train_cod = torch.utils.data.DataLoader(data_train_cod, batch_size=256, shuffle=True)

In [25]:
X_val = data_test.data.float() / 255
X_val = X_val.view(-1,784)
X_val = X_val.to(device)

In [26]:
with torch.no_grad():
  X_val_cod = model_enc(X_val)
cls_val = data_test.targets

In [27]:
X_val_cod.shape

torch.Size([10000, 32])

In [28]:
data_test_cod = torch.utils.data.TensorDataset(X_val_cod, cls_val)
loader_val_cod = torch.utils.data.DataLoader(data_test_cod, batch_size=256, shuffle=True)

## Etapa 7: Classificação sem redução de dimensionalidade

In [29]:
cls_1 = nn.Sequential(nn.Linear(784,397),
                      nn.ReLU(),
                      nn.Linear(397,10),
                      nn.LogSoftmax())

In [30]:
cls_1.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(cls_1.parameters())

In [32]:
for epoch in range(20):
  running_loss_train = 0.
  running_acc_train = 0.
  for data in loader_train:
    cls_1.train()
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    inputs = inputs.view(-1,28*28)

    optimizer.zero_grad()
    outputs = cls_1(inputs)
    ps = torch.exp(outputs)
    _, top_class = ps.topk(k=1, dim=1)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss_train += loss.item()
    running_acc_train += accuracy_score(labels.detach().cpu().numpy(),
                                        top_class.detach().cpu().numpy())
    
  
  running_loss_test = 0.
  running_acc_val = 0.
  for data in loader_test:
    cls_1.eval()
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    inputs = inputs.view(-1,28*28)
    
    outputs = cls_1(inputs)
    ps = torch.exp(outputs)
    _, top_class = ps.topk(k=1, dim=1)
    loss = criterion(outputs, labels)
    
    running_loss_val += loss.item()
    running_acc_val += accuracy_score(labels.detach().cpu().numpy(),
                                      top_class.detach().cpu().numpy())
    
  print('ÉPOCA {:3d}: PERDA TREINO {:.5f} - PRECISÃO TREINO {:5f} | PERDA VALIDAÇÃO {:.5f} - PRECISÃO VALIDAÇÃO {:5f}' \
        .format(epoch+1, running_loss_train/len(loader_train),
                running_acc_train/len(loader_train),
                running_loss_val/len(loader_test),
                running_acc_val/len(loader_test)))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


ÉPOCA   1: PERDA TREINO 0.01251 - PRECISÃO TREINO 0.997557 | PERDA VALIDAÇÃO 1.39427 - PRECISÃO VALIDAÇÃO 0.980371
ÉPOCA   2: PERDA TREINO 0.01083 - PRECISÃO TREINO 0.998072 | PERDA VALIDAÇÃO 1.45822 - PRECISÃO VALIDAÇÃO 0.980469
ÉPOCA   3: PERDA TREINO 0.00864 - PRECISÃO TREINO 0.998604 | PERDA VALIDAÇÃO 1.52449 - PRECISÃO VALIDAÇÃO 0.979688
ÉPOCA   4: PERDA TREINO 0.00646 - PRECISÃO TREINO 0.999202 | PERDA VALIDAÇÃO 1.59472 - PRECISÃO VALIDAÇÃO 0.979102
ÉPOCA   5: PERDA TREINO 0.00539 - PRECISÃO TREINO 0.999302 | PERDA VALIDAÇÃO 1.66207 - PRECISÃO VALIDAÇÃO 0.981543
ÉPOCA   6: PERDA TREINO 0.00507 - PRECISÃO TREINO 0.999252 | PERDA VALIDAÇÃO 1.73423 - PRECISÃO VALIDAÇÃO 0.980078
ÉPOCA   7: PERDA TREINO 0.00389 - PRECISÃO TREINO 0.999584 | PERDA VALIDAÇÃO 1.80596 - PRECISÃO VALIDAÇÃO 0.980469
ÉPOCA   8: PERDA TREINO 0.00321 - PRECISÃO TREINO 0.999734 | PERDA VALIDAÇÃO 1.87378 - PRECISÃO VALIDAÇÃO 0.981543
ÉPOCA   9: PERDA TREINO 0.00238 - PRECISÃO TREINO 0.999934 | PERDA VALIDAÇÃO 1.9

## Etapa 8: Classificação com redução de dimensionalidade

In [33]:
cls_2 = nn.Sequential(
    nn.Linear(32,21),
    nn.ReLU(),
    nn.Linear(21,10),
    nn.LogSoftmax()
)

In [35]:
cls_2.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(cls_2.parameters())

In [38]:
for epoch in range(20):
  running_loss_train = 0.
  running_acc_train = 0.
  for data in loader_train_cod:
    cls_2.train()
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = cls_2(inputs)
    ps = torch.exp(outputs)
    _, top_class = ps.topk(k=1, dim=1)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss_train += loss.item()
    running_acc_train += accuracy_score(labels.detach().cpu().numpy(),
                                        top_class.detach().cpu().numpy())
    
  
  running_loss_test = 0.
  running_acc_val = 0.
  for data in loader_val_cod:
    cls_2.eval()
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    
    outputs = cls_2(inputs)
    ps = torch.exp(outputs)
    _, top_class = ps.topk(k=1, dim=1)
    loss = criterion(outputs, labels)
    
    running_loss_val += loss.item()
    running_acc_val += accuracy_score(labels.detach().cpu().numpy(),
                                      top_class.detach().cpu().numpy())
    
  print('ÉPOCA {:3d}: PERDA TREINO {:.5f} - PRECISÃO TREINO {:5f} | PERDA VALIDAÇÃO {:.5f} - PRECISÃO VALIDAÇÃO {:5f}' \
        .format(epoch+1, running_loss_train/len(loader_train),
                running_acc_train/len(loader_train),
                running_loss_val/len(loader_test),
                running_acc_val/len(loader_test)))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


ÉPOCA   1: PERDA TREINO 0.56218 - PRECISÃO TREINO 0.838697 | PERDA VALIDAÇÃO 3.23756 - PRECISÃO VALIDAÇÃO 0.878613
ÉPOCA   2: PERDA TREINO 0.42664 - PRECISÃO TREINO 0.873881 | PERDA VALIDAÇÃO 3.60756 - PRECISÃO VALIDAÇÃO 0.890332
ÉPOCA   3: PERDA TREINO 0.38372 - PRECISÃO TREINO 0.885849 | PERDA VALIDAÇÃO 3.96128 - PRECISÃO VALIDAÇÃO 0.895508
ÉPOCA   4: PERDA TREINO 0.36144 - PRECISÃO TREINO 0.892055 | PERDA VALIDAÇÃO 4.28803 - PRECISÃO VALIDAÇÃO 0.901953
ÉPOCA   5: PERDA TREINO 0.34467 - PRECISÃO TREINO 0.896260 | PERDA VALIDAÇÃO 4.60722 - PRECISÃO VALIDAÇÃO 0.903809
ÉPOCA   6: PERDA TREINO 0.33244 - PRECISÃO TREINO 0.899939 | PERDA VALIDAÇÃO 4.91879 - PRECISÃO VALIDAÇÃO 0.906055
ÉPOCA   7: PERDA TREINO 0.32159 - PRECISÃO TREINO 0.903480 | PERDA VALIDAÇÃO 5.21671 - PRECISÃO VALIDAÇÃO 0.910742
ÉPOCA   8: PERDA TREINO 0.31317 - PRECISÃO TREINO 0.904555 | PERDA VALIDAÇÃO 5.51260 - PRECISÃO VALIDAÇÃO 0.911133
ÉPOCA   9: PERDA TREINO 0.30583 - PRECISÃO TREINO 0.907530 | PERDA VALIDAÇÃO 5.7